In [ ]:
# prompt: load drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install xmltodict opencv-python matplotlib numpy
import os
import cv2
import xmltodict
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive



In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# Paths
image_folder = "/content/drive/MyDrive/cv_fir_smk/images"
label_folder = "/content/drive/MyDrive/cv_fir_smk/labels"

# Select a sample image
image_name = "Datacluster Fire and Smoke Sample (1).jpg"
image_path = os.path.join(image_folder, image_name)
label_path = os.path.join(label_folder, image_name.replace(".jpg", ".txt"))

# Load image
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Get image dimensions
h, w, _ = img.shape


In [ ]:
import os
import shutil
import random

# Define paths
dataset_path = "/content/drive/MyDrive/cv_fir_smk/dataset"
image_folder = "/content/drive/MyDrive/cv_fir_smk/images"
label_folder = "/content/drive/MyDrive/cv_fir_smk/labels"

train_img_dir = os.path.join(dataset_path, "images/train")
val_img_dir = os.path.join(dataset_path, "images/val")
train_lbl_dir = os.path.join(dataset_path, "labels/train")
val_lbl_dir = os.path.join(dataset_path, "labels/val")

# Create directories if they don't exist
for path in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    os.makedirs(path, exist_ok=True)

# Get all image names
all_images = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]

# Shuffle and split dataset (80% train, 20% val)
random.shuffle(all_images)
split_idx = int(0.8 * len(all_images))
train_images = all_images[:split_idx]
val_images = all_images[split_idx:]

# Move files to respective folders
for img_name in train_images:
    shutil.copy(os.path.join(image_folder, img_name), train_img_dir)
    shutil.copy(os.path.join(label_folder, img_name.replace(".jpg", ".txt")), train_lbl_dir)

for img_name in val_images:
    shutil.copy(os.path.join(image_folder, img_name), val_img_dir)
    shutil.copy(os.path.join(label_folder, img_name.replace(".jpg", ".txt")), val_lbl_dir)

print(" Dataset organized successfully!")


In [ ]:

# Define dataset paths
dataset_path = "/content/drive/MyDrive/cv_fir_smk/dataset"
train_img_dir = os.path.join(dataset_path, "images/train")
val_img_dir = os.path.join(dataset_path, "images/val")

# Define the YAML content
yaml_content = f"""
train: {train_img_dir}
val: {val_img_dir}

nc: 2  # Number of classes (update if different)
names: ["fire", "smoke"]
"""

# Save the YAML file
yaml_path = os.path.join(dataset_path, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f" data.yaml file created at: {yaml_path}")


In [ ]:
!cat /content/drive/MyDrive/cv_fir_smk/dataset/data.yaml



train: /content/drive/MyDrive/cv_fir_smk/dataset/images/train
val: /content/drive/MyDrive/cv_fir_smk/dataset/images/val

nc: 2  # Number of classes (update if different)
names: ["fire", "smoke"]


In [ ]:

# Import required libraries
import torch
from ultralytics import YOLO

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the YOLOv8 model (medium version for better accuracy)
model = YOLO("yolov8m.pt")

# Define training parameters
train_params = {
    "data": "/content/drive/MyDrive/cv_fir_smk/dataset/data.yaml",  # Dataset path
    "epochs": 10,  # Increase epochs for better accuracy
    "imgsz": 640,  # Image size
    "batch": 16,  # Adjust batch size based on GPU memory
    "device": device,  # Auto-detect GPU/CPU
    "optimizer": "AdamW",  # Better optimizer than default SGD
    "lr0": 0.001,  # Initial learning rate
    "lrf": 0.1,  # Final learning rate
    "momentum": 0.9,  # Momentum for optimizer
    "weight_decay": 0.0005,  # Regularization for better generalization
    "degrees": 10,  # Data augmentation: rotation (±10°)
    "translate": 0.1,  # Data augmentation: translation
    "scale": 0.5,  # Data augmentation: scaling
    "shear": 10,  # Data augmentation: shearing
    "hsv_h": 0.015,  # HSV augmentation (hue)
    "hsv_s": 0.7,  # HSV augmentation (saturation)
    "hsv_v": 0.4,  # HSV augmentation (value)
    "patience": 20,  # Early stopping if no improvement
    "cache": True,  # Cache dataset for faster training
    "workers": 4,  # Number of dataloader workers
}

# Train the model
model.train(**train_params)


Using device: cuda
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/cv_fir_smk/dataset/data.yaml, epochs=10, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=cuda, workers=4, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

train: Scanning /content/drive/MyDrive/cv_fir_smk/dataset/labels/train... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [02:27<00:00,  1.85s/it]

train: WARNING ⚠️ /content/drive/MyDrive/cv_fir_smk/dataset/images/train/Datacluster Fire and Smoke Sample (22).jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/cv_fir_smk/dataset/images/train/Datacluster Fire and Smoke Sample (23).jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/cv_fir_smk/dataset/images/train/Datacluster Fire and Smoke Sample (26).jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/cv_fir_smk/dataset/images/train/Datacluster Fire and Smoke Sample (29).jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/cv_fir_smk/dataset/images/train/Datacluster Fire and Smoke Sample (31).jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/cv_fir_smk/dataset/images/train/Datacluster Fire and Smoke Sample (32).jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/cv_fir_smk/dataset/images/train/Datacluster Fire and Smoke Sample

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.1GB RAM): 100%|██████████| 80/80 [00:09<00:00,  8.56it/s]


...

                   all         20         23      0.836      0.444       0.52      0.252



10 epochs completed in 0.015 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 52.0MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.27it/s]


                   all         20         23      0.799      0.478       0.52      0.252
                  fire         19         23      0.799      0.478       0.52      0.252
Speed: 0.2ms preprocess, 10.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7daddd36b6d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 